
## Imports

The above imports are not at all mandatory in order to have the following code work. This cell is here to ensure that your setup is correct. 

In [5]:
%cd ..

%load_ext autoreload
%autoreload 2

/home/arthur/drl/mva-drl-project


## DDPG Agent on pendulum 


In [49]:
import acme

from src.environments.inverted_pendulum import InvertedPendulumEnv, inverted_pendulum_env_factory
from src.agents import DDPGAgent
from src.interaction_loops import ddpg_parallel_interaction_loop, evaluation_parallel_interaction_loop

In [57]:
pendulum_environment = InvertedPendulumEnv(for_evaluation=True)  #### WARNING, I changed the env
pendulum_environment_spec = acme.make_environment_spec(pendulum_environment)

In [62]:
# define the agent
max_learner_steps=100000
pendulum_agent = DDPGAgent(seed=0, 
actor_learning_rate=5e-4,
critic_learning_rate=5e-4 ,
gamma=.95, 
environment_spec=pendulum_environment_spec, tau=1)

In [63]:
ddpg_parallel_interaction_loop(pendulum_agent, env_factory=inverted_pendulum_env_factory, max_learner_steps=max_learner_steps, num_actors=16, batch_size=32, buffer_size=1000)


ValueError: Expected dict, got None.

In [39]:
self = pendulum_agent

self._rng, init_rng = jax.random.split(self._rng)
learner_state = self._init_fn(init_rng, self._generate_dummy_transition())


In [54]:
pendulum_agent._learner_state.state

dict_keys(['policy/batch_norm/~/mean_ema', 'policy/batch_norm/~/var_ema', 'policy/batch_norm_1/~/mean_ema', 'policy/batch_norm_1/~/var_ema', 'policy/batch_norm_2/~/mean_ema', 'policy/batch_norm_2/~/var_ema', 'policy_target/batch_norm/~/mean_ema', 'policy_target/batch_norm/~/var_ema', 'policy_target/batch_norm_1/~/mean_ema', 'policy_target/batch_norm_1/~/var_ema', 'policy_target/batch_norm_2/~/mean_ema', 'policy_target/batch_norm_2/~/var_ema', 'value/batch_norm/~/mean_ema', 'value/batch_norm/~/var_ema', 'value/batch_norm_1/~/mean_ema', 'value/batch_norm_1/~/var_ema', 'value_target/batch_norm/~/mean_ema', 'value_target/batch_norm/~/var_ema', 'value_target/batch_norm_1/~/mean_ema', 'value_target/batch_norm_1/~/var_ema'])

In [31]:
import jax

In [ ]:
tree_map(lambda x,y:x+y, pendulum_agent._learner_state.params, pendulum_agent._learner_state.params)

In [ ]:
for _ in range(10):
  ddpg_parallel_interaction_loop(pendulum_agent, env_factory=inverted_pendulum_env_factory, max_learner_steps=max_learner_steps, sequence_length=sequence_length, num_actors=16)
  plot_policy_on_pendulum(pendulum_agent, 50)
  evaluation_parallel_interaction_loop(pendulum_agent, env_factory=inverted_pendulum_env_factory, sequence_length=200, num_actors=4)

In [ ]:
from src.networks import ValueNetworkDDPG, PolicyNetworkDDPG, PolicyNetwork
from src.data import Transition
import jax.numpy as jnp
import jax
import haiku as hk

In [ ]:
from src.data import Transition
observation = pendulum_environment_spec.observations.generate_value()
action = pendulum_environment_spec.actions.generate_value()

tr = Transition(
    obs_tm1= observation[None],
    action_tm1= action[None],
    reward_t= jnp.zeros(1)[None],
    discount_t=jnp.zeros(1)[None],
    obs_t=observation[None],
    done=jnp.zeros(1)[None])

In [ ]:
jnp.concatenate([tr.obs_t, tr.action_tm1], axis=1)

In [ ]:
def f(transition):
    action = PolicyNetworkDDPG(environment_spec.actions)(transition.obs_tm1, True)
    return action

def f1(obs):
    return hk.BatchApply(PolicyNetworkDDPG(environment_spec.actions))(obs, True)
    
init_loss, apply_loss = hk.without_apply_rng(hk.transform_with_state(f))

In [ ]:
rng = jax.random.PRNGKey(seed=0)

In [ ]:
init_loss(rng,tr)

In [ ]:
tr.obs_tm1